In [17]:
import requests
import csv

In [8]:
# Connection strings
CLIENT_ID = "3t6hp09w7jck16tyvkho4p2rgtd9f3"
CLIENT_SECRET = "i6vg2u4i4v631fo3nfdrui5wdxyvx7"

In [9]:
# Getting OAuth token
def get_app_token():
    url = "https://id.twitch.tv/oauth2/token"
    params = {
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
        "grant_type": "client_credentials"
             }
    response = requests.post(url, data=params)
    response.raise_for_status()
    return response.json()["access_token"]    
access_token = get_app_token()
print("Access token :", access_token)

Access token : 8ipxowvkrjlga31dktt0spdq7jgcob


In [15]:
CHANNEL_LOGIN = "BLASTPremier"
OUTPUT_CSV = f"/Users/hari14/Desktop/PHD/twitch_emotes_analytics_project/data/videos_on_demand_urls/{CHANNEL_LOGIN}_vods.csv"
print(CHANNEL_LOGIN,OUTPUT_CSV)

BLASTPremier /Users/hari14/Desktop/PHD/twitch_emotes_analytics_project/data/videos_on_demand_urls/BLASTPremier_vods.csv


In [11]:
# --- Step 2: Get User ID ---
def get_user_id(access_token):
    headers = {
        "Client-ID": CLIENT_ID,
        "Authorization": f"Bearer {access_token}"
    }
    response = requests.get(
        "https://api.twitch.tv/helix/users",
        headers=headers,
        params={"login": CHANNEL_LOGIN}
    )
    response.raise_for_status()
    return response.json()["data"][0]["id"]

In [12]:
# --- Step 3: Fetch All Archived VODs ---
def get_all_vods(user_id, access_token):
    vods = []
    url = "https://api.twitch.tv/helix/videos"
    headers = {
        "Client-ID": CLIENT_ID,
        "Authorization": f"Bearer {access_token}"
    }
    params = {
        "user_id": user_id,
        "type": "archive",
        "first": 100  # max per page
    }

    while True:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()
        vods.extend(data["data"])

        if "pagination" in data and "cursor" in data["pagination"]:
            params["after"] = data["pagination"]["cursor"]
        else:
            break

    return vods

In [13]:
# --- Step 4: Save to CSV ---
def save_vods_to_csv(vods):
    with open(OUTPUT_CSV, mode='w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=["id", "title", "created_at", "duration", "url"])
        writer.writeheader()
        for v in vods:
            writer.writerow({
                "id": v["id"],
                "title": v["title"],
                "created_at": v["created_at"],
                "duration": v["duration"],
                "url": v["url"]
            })

In [18]:
# --- MAIN EXECUTION ---
if __name__ == "__main__":
    print("🔐 Getting OAuth token...")
    token = get_app_token()

    print("🎯 Fetching user ID for channel:", CHANNEL_LOGIN)
    user_id = get_user_id(token)

    print("📺 Fetching all archived VODs...")
    vods = get_all_vods(user_id, token)
    print(f"✅ Found {len(vods)} VODs")

    print(f"💾 Saving to CSV: {OUTPUT_CSV}")
    save_vods_to_csv(vods)
    print("🎉 Done!")

🔐 Getting OAuth token...
🎯 Fetching user ID for channel: BLASTPremier
📺 Fetching all archived VODs...
✅ Found 19 VODs
💾 Saving to CSV: /Users/hari14/Desktop/PHD/twitch_emotes_analytics_project/data/videos_on_demand_urls/BLASTPremier_vods.csv
🎉 Done!


In [19]:
import pandas as pd

In [20]:
df = pd.read_csv(OUTPUT_CSV)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          19 non-null     int64 
 1   title       19 non-null     object
 2   created_at  19 non-null     object
 3   duration    19 non-null     object
 4   url         19 non-null     object
dtypes: int64(1), object(4)
memory usage: 892.0+ bytes


In [22]:
df

,id,title,created_at,duration,url
0,2493025305,"BLAST.tv Austin Major 2025, Showmatch - Team U...",2025-06-22T16:35:24Z,6h26m37s,https://www.twitch.tv/videos/2493025305
1,2492141783,"BLAST.tv Austin Major 2025, Semi-Finals - MOUZ...",2025-06-21T16:49:38Z,7h12m44s,https://www.twitch.tv/videos/2492141783
2,2491219390,"BLAST.tv Austin Major 2025, Quarter-Finals, Da...",2025-06-20T16:45:26Z,7h12m55s,https://www.twitch.tv/videos/2491219390
3,2490363560,"BLAST.tv Austin Major 2025, Quarterfinal, Day ...",2025-06-19T17:34:04Z,10h32m5s,https://www.twitch.tv/videos/2490363560
4,2486728026,"BLAST.tv Austin Major 2025, Stage 3, Day 4 - S...",2025-06-15T14:09:40Z,11h18m28s,https://www.twitch.tv/videos/2486728026
5,2485824420,"BLAST.tv Austin Major 2025, Stage 3, Day 3 - S...",2025-06-14T14:12:02Z,9h12m32s,https://www.twitch.tv/videos/2485824420
6,2484895485,"BLAST.tv Austin Major 2025, Stage 3, Day 2 - S...",2025-06-13T14:18:14Z,8h21m30s,https://www.twitch.tv/videos/2484895485
7,2484020698,"BLAST.tv Austin Major 2025, Stage 3, Day 1 - S...",2025-06-12T14:18:19Z,9h37m36s,https://www.twitch.tv/videos/2484020698
8,2482259774,"BLAST.tv Austin Major 2025, Stage 2, Day 4 - S...",2025-06-10T14:11:52Z,8h49m7s,https://www.twitch.tv/videos/2482259774
9,2481394555,"BLAST.tv Austin Major 2025, Stage 2, Day 2 - S...",2025-06-09T14:11:06Z,10h46m41s,https://www.twitch.tv/videos/2481394555


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          19 non-null     int64 
 1   title       19 non-null     object
 2   created_at  19 non-null     object
 3   duration    19 non-null     object
 4   url         19 non-null     object
dtypes: int64(1), object(4)
memory usage: 892.0+ bytes
